In [ ]:
import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
import rpy2.rinterface_lib.callbacks
import anndata2ri
import pkg_resources
import logging
#import sc_toolbox
%load_ext rpy2.ipython
import numpy as np
import scanpy as sc
import seaborn as sns
import warnings
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helper_edgeR as heR
print(sys.executable)
def return_local_paths(drive,pre,add_path = True):
    #drive: e.g. "F" - identifier for external drive
    #pre: unique script identifier, e.g. 'M10'
    #add_path: True oder 
    base_model_path = os.path.join(drive + ":\\","monkey_IZI","analysis","models",pre)
    base_table_path = os.path.join(drive + ":\\","monkey_IZI","analysis","tables",pre)
    base_plots_path = os.path.join(drive + ":\\","monkey_IZI","analysis","plots",pre)
    base_anndata_objects = os.path.join(drive + ":\\","monkey_IZI","analysis","anndata_objects",pre)
    paths = [base_model_path,base_table_path,base_plots_path,base_anndata_objects]
    if add_path:
        for path in paths:
            if not os.path.exists(path):
                os.mkdir(path)
    return base_model_path,base_table_path,base_plots_path,base_anndata_objects

In [ ]:
pre = "MH87"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

In [ ]:
_,base_table_path_MH86,_,_ = return_local_paths(drive = drive,pre = "MH86",add_path = True)

In [ ]:
celltypes = ['B','CD14_Mono','CD4_T','CD8_T','MAIT','MAIT','NKProliferating']
comps_M = ['06hr','24hr']
comps_H = ['06hr','24hr']

In [ ]:
dfs = []
for celltype in celltypes:
    for comp_M in comps_M:
        for comp_H in comps_H:
            df = pd.read_csv(os.path.join(base_table_path_MH86,'MH86_' + str(celltype) + '_M_' + comp_M + '_H_' + comp_H + '.csv'),index_col=0)
            df['celltype'] = celltype
            df['comp_M'] = comp_M
            df['comp_H'] = comp_H
            dfs.append(df)
dfs_all = pd.concat(dfs)

In [ ]:
dfs_all.to_csv(os.path.join(base_table_path,pre + '_dfs_all.csv'))

In [ ]:
%%R -i dfs_all -o results_df
results_df <- data.frame(
  celltype = character(),
  comp_M = character(),
  comp_H = character(),
  matching_sign = numeric(),
  no_matching_sign = numeric(),
  nr_genes = numeric(),
  p_value = numeric(),
  stringsAsFactors = FALSE
)
for (celltype in unique(dfs_all$celltype)) {
    for (comp_m in unique(dfs_all$comp_M)) {
        for (comp_h in unique(dfs_all$comp_H)) {
            data_ct_comp <- dfs_all[dfs_all$celltype == celltype & dfs_all$comp_M == comp_m & dfs_all$comp_H == comp_h, ]
            matching_sign <- sum(data_ct_comp$logFC_M == data_ct_comp$logFC_H)
            no_matching_sign <- sum(data_ct_comp$logFC_M != data_ct_comp$logFC_H)
            nr_genes <- nrow(data_ct_comp)
            # Count where values are different
            #df = df[,-1]
            res <- prop.test(x=matching_sign,n = nr_genes,alternative = "two.sided",conf.level = 0.95)
            #print(res)
            
            results_df <- rbind(results_df, data.frame(
          celltype = celltype,
          comp_M = comp_m,
          comp_H = comp_h,
          matching_sign = matching_sign,
          no_matching_sign = no_matching_sign,
          nr_genes = nr_genes,
          p_value = res$p.value
        ))
    }
    }
}

In [ ]:
results_df.rename(columns={'p_value': 'p_value_prop_test'}, inplace=True)
results_df.to_csv(os.path.join(base_table_path,pre + '_results_df_prop_test.csv'))

In [ ]:
def return_package_version_local_path(drive):
    return os.path.join(drive + ":\\","monkey_IZI","analysis","package_versions")

def save_package_versions(base_package_version_path,pre,do_print = True):
    #base_package_version_path: path to folder where to save all package versions
    #pre: unique script/notebook identifier, e.g. 'M10'
    with open(os.path.join(base_package_version_path, pre + '_package_versions.txt'), "w") as file:
        for package in pkg_resources.working_set:
            file.write(f"{package.key}=={package.version}\n")
            if do_print:
                print(f"{package.key}=={package.version}")
def print_main_versions():
    sc.logging.print_header()

base_package_version_path = return_package_version_local_path(drive=drive)
save_package_versions(base_package_version_path,pre,do_print = True)
print_main_versions()